In [32]:
import pandas as pd
import sqlite3
from autocorrect import Speller
import re

In [84]:
# Подключение к базе данных
db_path = '../db/data.db'
conn = sqlite3.connect(db_path)

In [128]:
# Загрузка данных из каждой таблицы в отдельный DataFrame
person_df = pd.read_sql("SELECT * FROM person", conn)
education_df = pd.read_sql("SELECT * FROM education", conn)
jobs_df = pd.read_sql("SELECT * FROM jobs", conn)

def to_lower(text: str) -> str:
    return text.lower()

def cut(text: str) -> str:
    symbols_for_cut = [",", "(", ";"]
    for symbol in symbols_for_cut:
        idx = text.find(symbol)
        if idx != -1:
            text = text[:idx]
    return text

jobs_df['job_name'] = jobs_df['job_name'].apply(to_lower)
jobs_df['job_name'] = jobs_df['job_name'].apply(cut)

In [129]:
spell = Speller("ru")

def correct(text: str) -> str:
    return str(spell(text))

jobs_df['job_name'] = jobs_df['job_name'].apply(correct)

In [130]:
pd.set_option('display.max_rows', 10)

jobs_df['job_name'] = jobs_df['job_name'].str.replace(".", " ")
jobs_df['job_name'] = jobs_df['job_name'].str.replace("  ", " ")

# Словарь для расшифровки сокращений
abbreviation_dict = {
    "вет": "ветеринарный",
    "гл": "главный",
    "ген": "генеральный",
    "зав": "заведующий",
    "зам": "заместитель",
    "мех": "механик",
    "нач": "начальник",
    "тех": "технический",
    "ст": "старший",
    "кат": "категория",
    "кл": "класс",
    "и о": "исполняющий обязанности",
    "мтк": "медицинско-технический контроль",
}

# Функция для расшифровки сокращений в названиях должностей
def expand_abbreviations(job_name, abbr_dict=abbreviation_dict):
    job_name += " "
    for abbr, full_form in abbr_dict.items():
        job_name = job_name.replace(abbr + " ", full_form + " ")
    return job_name.strip()

jobs_df['job_name'] = jobs_df['job_name'].apply(expand_abbreviations)

def remove_reducant_space(text: str) -> str:
    return text.strip()

jobs_df['job_name'] = jobs_df['job_name'].apply(remove_reducant_space)

jobs_df

,id,job_name,person_id,desc,start,end
0,1,ветеринарный врач,1,Планирование и проведение ветеринарных лечебно...,Август 2018,None
1,2,ветеринарный врач,1,Ветеринарный врач,Август 2018,Ноябрь 2019
2,3,тракторист-машинистарший,2,-работа на тракторе Джон-Дир 8020\n-ТО и ремон...,Август 2014,Октябрь 2014
3,4,тракторист-машинистарший,2,-оператор з/у комбайна Джон Дир\n-ремонт и ТО:...,Июль 2013,Сентябрь 2013
4,5,агроном,3,-,Сентябрь 2018,None
...,...,...,...,...,...,...
4330,4331,старший инженер,1000,"Организация работы. эксплуатация , техническое...",Август 1976,None
4331,4332,зоотехник,1001,Контроль поедаемости кормов животными.\nКоррек...,Август 2020,None
4332,4333,зоотехник,1001,Контроль кормления коров.\nЗаготовка сена.\nПр...,Декабрь 2019,Август 2020
4333,4334,помощник зоотехника по кормлению,1001,Контроль поедаемости кормов животными.\nКоррек...,Сентябрь 2019,Ноябрь 2019


In [94]:
jobs_df['job_name']

0                      ветеринарный врач
1                      ветеринарный врач
2               тракторист-машинистарший
3               тракторист-машинистарший
4                                агроном
                      ...               
4330                     старший инженер
4331                           зоотехник
4332                           зоотехник
4333    помощник зоотехника по кормлению
4334                 помощник зоотехника
Name: job_name, Length: 4335, dtype: object

In [98]:
from collections import Counter

words = jobs_df['job_name'].str.split().explode()

# Подсчитать частоту каждого слова
word_counts = Counter(words)

# Вывести самые частые слова
most_common_words = word_counts.most_common()

# Печать результатов
print(most_common_words)

[('по', 477), ('главный', 461), ('ветеринарный', 421), ('врач', 402), ('агроном', 354), ('инженер', 305), ('механик', 215), ('начальник', 206), ('и', 154), ('отдела', 138), ('специалистарший', 138), ('менеджер', 138), ('директор', 127), ('зоотехник', 126), ('заместитель', 122), ('старший', 116), ('мастер', 111), ('машинистарший', 107), ('водитель', 105), ('участка', 89), ('директора', 84), ('заведующий', 83), ('технолог', 83), ('ведущий', 80), ('оператор', 73), ('инженер-механик', 73), ('руководитель', 72), ('разряда', 66), ('тракторист-машинистарший', 63), ('управляющий', 55), ('цеха', 53), ('службы', 50), ('оборудования', 49), ('производства', 49), ('слесарь', 47), ('ремонту', 47), ('помощник', 45), ('сервисный', 44), ('продажам', 42), ('с', 38), ('-', 37), ('трактористарший', 35), ('механизатор', 33), ('отделения', 32), ('в', 32), ('врача', 32), ('бригадир', 31), ('сотрудник', 31), ('эксплуатации', 31), ('научный', 30), ('погрузчика', 30), ('региональный', 30), ('представитель', 30)

In [99]:
pd.set_option('display.max_rows', None)
jobs_df['job_name'].value_counts()
len(jobs_df['job_name'].unique())

1995